# Text-to-SQL solution on top of Amazon products

In this notebook we will explore what it takes to implement a text-to-SQL solution on top of our Amazon dataset using LlamaIndex.

## Imports

In [1]:
import openai
import pandas as pd

from superlinked_app.config import settings

settings.validate_processed_dataset_exists()

2025-01-09 17:39:10.764 | INFO     | superlinked_app.config:<module>:9 - Loading '.env' file from: /Users/pauliusztin/Documents/01_projects/hands-on-retrieval/.env


In [2]:
openai.api_key = settings.OPENAI_API_KEY

## Create Database Schema

In [3]:
from sqlalchemy import (
    Column,
    Float,
    Integer,
    MetaData,
    String,
    Table,
    create_engine,
    insert,
    select,
)

In [4]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

Create the product SQL table:

In [5]:
product_table_name = "product"
product_table = Table(
    product_table_name,
    metadata_obj,
    Column("asin", String(255), primary_key=True),
    Column("type", String(255)),
    Column("title", String(255)),
    Column("description", String(1000)),
    Column("review_rating", Float),
    Column("review_count", Integer),
    Column("price", Float),
)
metadata_obj.create_all(engine)

## Define SQL Database

We first define our SQLDatabase abstraction (a light wrapper around SQLAlchemy).

In [6]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI

In [7]:
llm = OpenAI(temperature=0.1, model=settings.OPENAI_MODEL_ID)

In [8]:
sql_database = SQLDatabase(engine, include_tables=[product_table_name])

Add the data to the SQL database:

In [9]:
df = pd.read_json(settings.PROCESSED_DATASET_PATH, lines=True).drop(columns=['category'])
df.head()

,asin,type,title,description,price,review_rating,review_count
0,B07WP4RXHY,product,YUEPIN U-Tube Clamp 304 Stainless Steel Hose P...,Product Description Specification: Material: 3...,9.99,4.7,54
1,B07VRZTK2N,product,"Apron for Women, Waterproof Adjustable Bib Coo...",,11.99,4.0,152
2,B07V2F5SN1,product,DIY 5D Diamond Painting by Number Kit for Adul...,Product Description 5D DIY Diamond Painting is...,9.99,4.6,378
3,B00MNLQQ7K,product,"Design Toscano QM2787100 Darby, the Forest Faw...",,40.72,4.7,274
4,B089YD2KK5,product,Crocs Jibbitz 5-Pack Alien Shoe Charms | Jibbi...,From the brand Previous page Shop Crocs Collec...,9.99,4.7,0


In [10]:
for _, row in df.iterrows():
    stmt = insert(product_table).values(**row.to_dict())
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

## Query Database

Let's query the database to see that everything worked fine.

In [11]:
stmt = select(
    product_table.c.asin,
    product_table.c.type,
    product_table.c.title,
    product_table.c.description,
    product_table.c.review_rating,
    product_table.c.review_count,
    product_table.c.price,
).select_from(product_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchmany(3)
    for result in results:
        print(f"ASIN: {result.asin}")
        print(f"Type: {result.type}")
        print(f"Title: {result.title}")
        print(f"Description: {result.description[:100]}...")  # Truncate long descriptions
        print(f"Rating: {result.review_rating:.1f}/5.0 ({result.review_count} reviews)")
        print(f"Price: ${result.price:.2f}")
        print("-" * 80)

ASIN: B07WP4RXHY
Type: product
Title: YUEPIN U-Tube Clamp 304 Stainless Steel Hose Pipe Cable Strap Clips With Rubber Cushioned (1-21/32"(42mm)-10pcs)
Description: Product Description Specification: Material: 304 Stainless Steel,100% New Rubber Color: Silver Shape...
Rating: 4.7/5.0 (54 reviews)
Price: $9.99
--------------------------------------------------------------------------------
ASIN: B07VRZTK2N
Type: product
Title: Apron for Women, Waterproof Adjustable Bib Cooking Aprons with Pocket-2 Side Coral Velvet Towels Stitched Durable Pinstripe Aprons for Dishwashing, Baking, Grill, Restaurant Even Garden Craft
Description: ...
Rating: 4.0/5.0 (152 reviews)
Price: $11.99
--------------------------------------------------------------------------------
ASIN: B07V2F5SN1
Type: product
Title: DIY 5D Diamond Painting by Number Kit for Adult, Full Drill Crystal Rhinestone Embroidery Cross Stitch Diamond Embroidery Dotz Kit Home Wall Decor 15.8×11.8 Inch (Wolf)
Description: Product Descripti

## Text-to-SQL Retriever

Let's use the LlamaIndex SQL retriever to map text to SQL instructions and retrieve data from the SQLite table. 

In [12]:
from llama_index.core.retrievers import NLSQLRetriever
from llama_index.core.response.notebook_utils import display_source_node

nl_sql_retriever = NLSQLRetriever(
    sql_database,
    tables=[product_table_name],
    return_raw=False,
)

All the content is in the `metadata`:

In [13]:
def format_sql_results_to_df(results):
    # Convert SQL results to dictionary format for DataFrame
    data = []
    for result in results:
        data.append({
            'type': result.metadata.get("type"),
            'title': result.metadata.get("title"),
            'price': result.metadata.get("price"),
            'review_rating': result.metadata.get("review_rating")
        })
    
    # Create DataFrame with specific columns
    df = pd.DataFrame({
        'type': [d['type'] for d in data],
        'title': [d['title'] for d in data], 
        'price': [d['price'] for d in data],
        'review_rating': [d['review_rating'] for d in data]
    })
    
    return df

### Examples 1: Simple

Let's start with a simple query:

In [14]:
results = nl_sql_retriever.retrieve(
   "books with a price lower than 100 and a rating bigger than 4 (also show the type)"
)
format_sql_results_to_df(results[:3])

,type,title,price,review_rating
0,product,Paper Clever Party Unicorn Diaper Raffle Ticke...,9.99,5.0
1,product,Goton Bling Case for Apple Watch Case Series 7...,13.99,5.0
2,product,Tender Leaf Toys - Hungry Wooden Tortoise Shap...,30.59,5.0


### Examples 2: Specific categories 

Now, we will complicate the query:

In [15]:
results = nl_sql_retriever.retrieve(
   "psychology and mindfulness with a rating bigger than 4"
)
format_sql_results_to_df(results[:3])

,type,title,price,review_rating


In [16]:
results = nl_sql_retriever.retrieve(
   "Return the top items (along with their price) with the highest reviews rating on science"
)
format_sql_results_to_df(results[:3])

,type,title,price,review_rating


### Example 3: Titles or keywords

Let's make it even more complex:

In [17]:
results = nl_sql_retriever.retrieve(
   "Lord of the Rings"
)
format_sql_results_to_df(results[:3])

,type,title,price,review_rating
0,None,Funko POP! Movies: Lord of The Rings - Lurtz C...,42.99,4.8


### More examples

In [18]:
results = nl_sql_retriever.retrieve(
   "Return the top books (along with their price and rating) with the highest reviews rating and lowest price."
)
for n in results[:3]:
    display_source_node(n, show_source_metadata=True)

**Node ID:** 7f938eec-4fb4-4dc8-a4de-9bf55e64cab9<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'Choose: An Invitation to the Best Day Ever Adventure', 'price': 15.0, 'review_rating': 5.0}<br>

**Node ID:** c37f7809-d953-4df7-af56-bde09298ea3e<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': '33 Days to Morning Glory: A Do-It-Yourself Retreat In Preparation for Marian Consecration', 'price': 13.49, 'review_rating': 4.9}<br>

**Node ID:** c73007e5-400a-4666-b40c-7322fb799679<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'Mom Set Free - Bible Study Book: Good News for Moms Who are Tired of Trying to be Good Enough', 'price': 15.99, 'review_rating': 4.8}<br>

In [19]:
results = nl_sql_retriever.retrieve(
   "Return the top products (along with their price and review) about cats or dogs with a great price and review"
)
for n in results[:3]:
    display_source_node(n, show_source_metadata=True)

**Node ID:** d0be1de8-5fba-442f-8613-3c53385c5835<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'TRUE CABLE Cat6 Riser (CMR), 1000ft, White, 23AWG 4 Pair Solid Bare Copper, 550MHz, ETL Listed, Unshielded Twisted Pair (UTP), Bulk Ethernet Cable', 'price': 195.99, 'review_rating': 4.8}<br>

**Node ID:** cad05da4-e87e-4273-a78b-e7501619b40e<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'PneumaticPlus SAU400A-N04G - Dual Air System 1/2" NPT - Air Filter, Regulator & Lubricator w/Additional Free Air Port 3/8" NPT - Manual Drain, Poly Bowl, Pressure Gauge', 'price': 159.99, 'review_rating': 4.8}<br>

**Node ID:** 6b9a99e4-a321-4125-afcd-9e9188b6fe2c<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'MAYASAF Catalytic Converter w/Gasket [1.8L Models Only] for Toyota Corolla/Matrix, Pontiac Vibe 2003 04 05 06 07 08 (EPA Compliant)', 'price': 129.99, 'review_rating': 3.9}<br>

In [20]:
results = nl_sql_retriever.retrieve(
   "I'm looking for a computer or laptop with a price bigger than 100 and a review bigger than 4"
)
for n in results[:3]:
    display_source_node(n, show_source_metadata=True)

## Plug in the RetrieverQueryEngine

We compose our SQL Retriever with our standard RetrieverQueryEngine to synthesize a response. The result is similar to LlamaIndex's `NLSQLTableQueryEngine` class, which does both retrieval and synthetization of the response.

In [21]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)

In [22]:
response = query_engine.query(
    "Return the top 5 books (along with their review count and price) with the highest reviews rating."
)

In [23]:
print(str(response))

Mom Set Free - Bible Study Book: Good News for Moms Who are Tired of Trying to be Good Enough - 781 reviews, $15.99  
Millennial Kosher: recipes reinvented for the modern palate - 466 reviews, $36.99  
Choose: An Invitation to the Best Day Ever Adventure - 62 reviews, $15.0  
33 Days to Morning Glory: A Do-It-Yourself Retreat In Preparation for Marian Consecration - 0 reviews, $13.49  
Narcotics Anonymous Step Working Guides - 0 reviews, $17.15


# Next Steps

Go to the **INSTALL_AND_USAGE** document to learn how to start the MongoDB vector index, Superlinked RESTful API and Streamlit frontend.